In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-extended/train.csv
/kaggle/input/titanic-extended/full.csv
/kaggle/input/titanic-extended/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/titanic-extended/full.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,WikiId,Name_wiki,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",Southampton,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",Cherbourg,"New York, New York, US",4,NaN,1.0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",Southampton,New York City,14?,NaN,3.0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",Southampton,"Scituate, Massachusetts, US",D,NaN,1.0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",Southampton,New York City,NaN,NaN,3.0


In [27]:
df[['Survived']].isnull().sum()

Survived    418
dtype: int64

In [28]:
df.shape

(1309, 21)

In [29]:
df.dropna(subset = ['Survived'], inplace=True)

In [30]:
df.shape

(891, 21)

In [31]:
X = df[['Age', 'Pclass', 'Fare']]
y = df[['Survived']]

In [32]:
X.isnull().sum()

Age       177
Pclass      0
Fare        0
dtype: int64

In [33]:
X.count()

Age       714
Pclass    891
Fare      891
dtype: int64

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer

In [34]:
X_knn = X.copy()

In [35]:
si = SimpleImputer(add_indicator = True)

In [36]:
X_imputed = si.fit_transform(X)

In [37]:
lr = LogisticRegression()

In [38]:
lr.fit(X_imputed, y)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [42]:
X_imputed.shape

(891, 4)

In [49]:
import warnings
warnings.filterwarnings("ignore")

In [50]:
print(np.mean(cross_val_score(lr, X_imputed, y.to_numpy(), scoring='accuracy', cv = 10 )))

0.7003995006242197


### we got 70 % accuracy with SI now lets try with KNN Imputer

In [52]:
X_knn.isnull().sum()

Age       177
Pclass      0
Fare        0
dtype: int64

In [101]:
knn = KNNImputer(n_neighbors = 2 , add_indicator=True)

In [102]:
X_imputer_knn = knn.fit_transform(X_knn)

In [103]:
lr = LogisticRegression()

In [104]:
lr.fit(X_imputer_knn, y)

LogisticRegression()

In [105]:
print(np.mean(cross_val_score(lr, X_imputer_knn, y.to_numpy(), scoring='accuracy', cv = 10 )))

0.7060174781523096


### Accuracy increased but not profitable

### 